## Autogen Discord Embedding

I'm to lazy to make an chroma db doc chunking and embedding function so just using an autogen agent that does that already

In [1]:
import os
import sys
import json
import tempfile
from pathlib import Path
sys.path.append('./')

import openai
import autogen
import chromadb

config_list = autogen.config_list_from_dotenv(dotenv_file_path='../../../.env')

In [2]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Create an instance of RetrieveAssistantAgent
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

client = chromadb.PersistentClient(path=f"{os.getcwd()}/chromadb")
path = Path('../data/docs/')

# Create an instance of RetrieveUserProxyAgent
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "default",
        "docs_path": str(path), 
        "chunk_token_size": 500,
        "model": config_list[0]["model"],
        "client": client, 
        "embedding_model": "all-MiniLM-L6-v2",
        "collection_name": "autogen-docs-qa"
    },
)


In [ ]:
# NOTE: Delete dir each instantiation
client.delete_collection('autogen-docs-qa')

In [6]:
# The collections don't get created until the chat is initated 😐
ragproxyagent.initiate_chat(assistant, problem="What is langchain?", clear_history=True) 

Trying to create collection.


INFO:autogen.retrieve_utils:Found 151 chunks.


doc_ids:  [['doc_90', 'doc_89', 'doc_92', 'doc_47', 'doc_70', 'doc_2', 'doc_84', 'doc_16', 'doc_133', 'doc_135', 'doc_40', 'doc_57', 'doc_96', 'doc_134', 'doc_53', 'doc_107', 'doc_78', 'doc_68', 'doc_129', 'doc_136']]
Adding doc_id doc_90 to context.
Adding doc_id doc_89 to context.
Adding doc_id doc_92 to context.
Adding doc_id doc_47 to context.
Adding doc_id doc_70 to context.
Adding doc_id doc_2 to context.
Adding doc_id doc_84 to context.
Adding doc_id doc_16 to context.
Adding doc_id doc_133 to context.
Adding doc_id doc_135 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_57 to context.
Adding doc_id doc_96 to context.
Adding doc_id doc_134 to context.
Adding doc_id doc_53 to context.
Adding doc_id doc_107 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user

In [5]:
ragproxyagent._retrieve_config['client'].list_collections()

[]

In [7]:
client.list_collections()

[Collection(name=autogen-docs-qa)]